# Dfsu 3D

Layered dfsu files comes in several different shapes:

* 3D
* 2D Vertical slice (transect)
* 1D Vertical profile

Two layer systems exist:

* sigma (terrain and surface following coordinates)
* sigma-z (sigma layers at the top and fixed z-layers at the bottom)

In sigma-layered files, all columns has the same number of layers. In sigma-z files, the number of z-layers can be different for different columns.  

Layered dfsu files have a "hidden" first dynamic item called "zn" with the (dynamic) z-positions of the nodes. 

Read the [MIKE IO dfsu documentation](https://dhi.github.io/mikeio/dfsu.html) for more info.

In [ ]:
import matplotlib.pyplot as plt
from mikeio import Dfsu

## 3D Sigma-z

In [ ]:
filename = "data/oresund_sigma_z.dfsu"
dfs = Dfsu(filename)
dfs

Apart from the normal dfsu properties, layered dfsu files have these properties: 

In [ ]:
print(f"Maximum number of layers: {dfs.n_layers}")
print(f"Number of sigma layers: {dfs.n_sigma_layers}")
print(f"Maximum number of z-layers: {dfs.n_z_layers}")
print(f"The layer number for each 3d element: {dfs.layer_ids}")
print(f"List of 3d element ids of surface layer: {dfs.top_elements}")
print(f"List of 3d element ids of bottom layer: {dfs.bottom_elements}")
print(f"List of number of layers for each column: {dfs.n_layers_per_column}")
print(f"The 2d-to-3d element connectivity table for a 3d object: {dfs.e2_e3_table[:3]} ...")
print(f"The associated 2d element id for each 3d element: {dfs.elem2d_ids}")

The associated 2D geometry for a 3D file can be outputted in this way:

In [ ]:
geom2d = dfs.geometry2d
geom2d

In [ ]:
geom2d.n_elements

### Top layer of 3D file

In [ ]:
elem_ids = dfs.top_elements
ds = dfs.read(elements=elem_ids)
print(ds)

In [ ]:
max_t = ds['Temperature'].max()
print(f'Maximum temperature in top layer: {max_t:.1f}C')

### Find position of max temperature and plot

Use numpy argmax() method to find the element with the largest value.

In [ ]:
timestep = 0
max_elem_id = ds['Temperature'][timestep,:].argmax()
top_element_coordinates = dfs.element_coordinates[dfs.top_elements]
max_x, max_y = top_element_coordinates[max_elem_id][:2]
max_x, max_y

In [ ]:
ax = dfs.plot(z=ds['Temperature'][timestep,:], figsize=(6,7), label="Temperature")
ax.plot(max_x, max_y, marker='*', markersize=20);

# Read 1D profile from 3D file

Find water column which has highest temperature and plot profile for all 3 time steps using static(!) z information. 

In [ ]:
elem_ids = dfs.find_nearest_profile_elements(max_x, max_y)
z_profile_static = dfs.element_coordinates[elem_ids,2]

In [ ]:
ds_profile = dfs.read(items=['Temperature'], elements=elem_ids)

In [ ]:
ds_profile['Temperature'].shape   # 3 timesteps and 4 layers (no z-layers at this position)

In [ ]:
for timestep in range(len(ds_profile.time)):
    plt.plot(ds_profile['Temperature'][timestep, :],z_profile_static, label=f"{ds_profile.time[timestep]}")
plt.title("Temperature profiles with static z")
plt.legend();

The profile can also be plotted using the dynamic z-values. We then need to read the z values in the nodes together with the temperature data for each time step. 

In [ ]:
ds_dyn = dfs.read(items=['Z coordinate','Temperature'], elements=elem_ids)

In [ ]:
print(f"Static z: {z_profile_static}")

# requires MIKE IO v0.8.1
# ec_dyn = dfs.get_element_coordinates(elements=elem_ids, zn=ds_dyn['Z coordinate'][0,:])
# print(f"Dynamic z: {ec_dyn[:,2].round(3)} (first time step)")
# ec_dyn = dfs.get_element_coordinates(elements=elem_ids, zn=ds_dyn['Z coordinate'][1,:])
# print(f"Dynamic z: {ec_dyn[:,2].round(3)} (second time step)")

In [ ]:
# requires MIKE IO v0.8.1
# for timestep in range(len(ds_dyn.time)):
#     ec_dyn = dfs.get_element_coordinates(elements=elem_ids, zn=ds_dyn['Z coordinate'][timestep,:])
#     plt.plot(ds_dyn['Temperature'][timestep, :],ec_dyn[:,2], label=f"{ds.time[timestep]}")
# plt.title("Temperature profiles with dynamic z")
# plt.legend();

## Read vertical slice

In [ ]:
filename = "data/oresund_vertical_slice.dfsu"
dfs = Dfsu(filename)
dfs

In [ ]:
print(dfs.bottom_elements[:9])
print(dfs.n_layers_per_column[:9])
print(dfs.top_elements[:9])

In [ ]:
ds = dfs.read(items="Temperature")

In [ ]:
dfs.plot_vertical_profile(ds[0][0,:], title="Transect", label="Temperature");